<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Diagnostic-methods" data-toc-modified-id="Diagnostic-methods-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Diagnostic methods</a></span><ul class="toc-item"><li><span><a href="#Finite-Time-Lyapunov-Exponent-(FTLE)" data-toc-modified-id="Finite-Time-Lyapunov-Exponent-(FTLE)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Finite Time Lyapunov Exponent (FTLE)</a></span></li><li><span><a href="#Polar-Rotation-Angle-(PRA)" data-toc-modified-id="Polar-Rotation-Angle-(PRA)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Polar Rotation Angle (PRA)</a></span><ul class="toc-item"><li><span><a href="#Lagrangian-vortex-boundary-from-PRA" data-toc-modified-id="Lagrangian-vortex-boundary-from-PRA-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Lagrangian vortex boundary from PRA</a></span></li></ul></li><li><span><a href="#Lagrangian-Averaged-Vorticity-Deviation-(LAVD)" data-toc-modified-id="Lagrangian-Averaged-Vorticity-Deviation-(LAVD)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Lagrangian Averaged Vorticity Deviation (LAVD)</a></span><ul class="toc-item"><li><span><a href="#Lagrangian-vortex-boundary-from-LAVD" data-toc-modified-id="Lagrangian-vortex-boundary-from-LAVD-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Lagrangian vortex boundary from LAVD</a></span></li></ul></li><li><span><a href="#Quasi-Objective-diagnostics" data-toc-modified-id="Quasi-Objective-diagnostics-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Quasi-Objective diagnostics</a></span><ul class="toc-item"><li><span><a href="#Trajectory-Rotation-Average-(TRA)" data-toc-modified-id="Trajectory-Rotation-Average-(TRA)-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Trajectory Rotation Average (TRA)</a></span></li><li><span><a href="#Trajectory-Rotation-Average-without-cancellations-$-(\overline{TRA})-$" data-toc-modified-id="Trajectory-Rotation-Average-without-cancellations-$-(\overline{TRA})-$-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Trajectory Rotation Average without cancellations $ (\overline{TRA}) $</a></span></li><li><span><a href="#Trajectory-Stretching-Exponent-$-(TSE)-$" data-toc-modified-id="Trajectory-Stretching-Exponent-$-(TSE)-$-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Trajectory Stretching Exponent $ (TSE) $</a></span></li><li><span><a href="#Trajectory-Stretching-Exponent-without-cancellations-$-(\overline{TSE})-$" data-toc-modified-id="Trajectory-Stretching-Exponent-without-cancellations-$-(\overline{TSE})-$-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Trajectory Stretching Exponent without cancellations $ (\overline{TSE}) $</a></span></li></ul></li></ul></li><li><span><a href="#Analytic-methods" data-toc-modified-id="Analytic-methods-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analytic methods</a></span><ul class="toc-item"><li><span><a href="#Hyperbolic-LCS" data-toc-modified-id="Hyperbolic-LCS-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hyperbolic LCS</a></span><ul class="toc-item"><li><span><a href="#Hyperbolic-LCS-from-local-variational-theory" data-toc-modified-id="Hyperbolic-LCS-from-local-variational-theory-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Hyperbolic LCS from local variational theory</a></span><ul class="toc-item"><li><span><a href="#Tensorlines---autonomous-differential-equation" data-toc-modified-id="Tensorlines---autonomous-differential-equation-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>Tensorlines - autonomous differential equation</a></span></li><li><span><a href="#Orientational-discontinuity" data-toc-modified-id="Orientational-discontinuity-2.1.1.2"><span class="toc-item-num">2.1.1.2&nbsp;&nbsp;</span>Orientational discontinuity</a></span></li><li><span><a href="#Steady-$-4^{th}-$-order-Runge---Kutta-Integration-handling-orientational-discontinuities" data-toc-modified-id="Steady-$-4^{th}-$-order-Runge---Kutta-Integration-handling-orientational-discontinuities-2.1.1.3"><span class="toc-item-num">2.1.1.3&nbsp;&nbsp;</span>Steady $ 4^{th} $-order Runge - Kutta Integration handling orientational discontinuities</a></span></li></ul></li></ul></li><li><span><a href="#Elliptic-LCS" data-toc-modified-id="Elliptic-LCS-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Elliptic LCS</a></span><ul class="toc-item"><li><span><a href="#Elliptic-LCS-from-local-variational-theory" data-toc-modified-id="Elliptic-LCS-from-local-variational-theory-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Elliptic LCS from local variational theory</a></span></li><li><span><a href="#Numerical-detection-of-singularities" data-toc-modified-id="Numerical-detection-of-singularities-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Numerical detection of singularities</a></span></li><li><span><a href="#Poincare-section" data-toc-modified-id="Poincare-section-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Poincare section</a></span></li></ul></li></ul></li><li><span><a href="#General-functions" data-toc-modified-id="General-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>General functions</a></span><ul class="toc-item"><li><span><a href="#Ridge-extraction" data-toc-modified-id="Ridge-extraction-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Ridge extraction</a></span></li><li><span><a href="#Local-extrema" data-toc-modified-id="Local-extrema-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Local extrema</a></span></li><li><span><a href="#Contourlines" data-toc-modified-id="Contourlines-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Contourlines</a></span></li><li><span><a href="#Area-of-polygon" data-toc-modified-id="Area-of-polygon-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Area of polygon</a></span></li><li><span><a href="#Convexity-deficiency" data-toc-modified-id="Convexity-deficiency-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Convexity deficiency</a></span></li><li><span><a href="#Point-inside-polygon" data-toc-modified-id="Point-inside-polygon-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Point inside polygon</a></span></li><li><span><a href="#Length-closed-material-line" data-toc-modified-id="Length-closed-material-line-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Length closed material line</a></span></li></ul></li></ul></div>

In [1]:
# LCS
import numpy as np
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

class LCS:
    
    def __init__(self):
        
        pass

<a id='Diagnostic_methods'></a>
# Diagnostic methods

<a id='FTLE'></a>
## Finite Time Lyapunov Exponent (FTLE)

The Finite Time Lyapunov Exponent (FTLE) is computed from the eigenvalues $ \lambda_i $ of the Cauchy-Green strain tensor $ C_{t_0}^{t_1}(\mathbf{x}_0) $:

\begin{equation}
FTLE = \dfrac{1}{2(t_1-t_0)}\log(\lambda_i), \quad i = 1, 2 
\end{equation}

where $ \lambda_i $ denotes the eigenvalue associated with the eigenvector $ \mathbf{\xi}_i(\mathbf{x}_0; t_0, t_1) $ of $ C_{t_0}^{t_1}(\mathbf{x}_0) $.
The minimum FTLE is computed for $ i = 1 $ (weakest eigenvalue), whereas the maximum FTLE is computed by setting $ i = 2 $ (dominant eigenvalue). If the type (minimum/maximum) of the FTLE is not specified, then we always assume that we are dealing with the maximum FTLE.

In [2]:
    def _FTLE_(self):
        
        if hasattr(self, 'C') == False:
        
            self._cauchy_green_strain()
    
        self.FTLE_min = np.zeros((self.len_Y, self.len_X))*np.nan
        self.FTLE_max = np.zeros((self.len_Y, self.len_X))*np.nan
        
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
                
                lambda_min, lambda_max, v_min, v_max = self._eigenvalues_and_eigenvectors(self.C[i, j, :, :])
                    
                self.FTLE_min[i, j] = 1/(2*(self.lenT))*np.log(lambda_min)
                self.FTLE_max[i, j] = 1/(2*(self.lenT))*np.log(lambda_max)
                        
        return self.FTLE_min, self.FTLE_max

<a id='PRA'></a>
## Polar Rotation Angle (PRA)

The Polar Rotation Angle (PRA) is computed from the eigenvectors $ \xi_i, \eta_i $ (with i = 1, 2) of the Cauchy-Green strain tensor $ C_{t_0}^{t_1}(\mathbf{x}_0) $:

\begin{equation}
PRA_{t_0}^{t_1}(\mathbf{x}_0) = \langle \xi_1(\mathbf{x}_0;t_0, t_1), \eta_1(\mathbf{x}_0;t_0, t_1) \rangle = \langle \xi_2(\mathbf{x}_0;t_0, t_1), \eta_2(\mathbf{x}_0;t_0, t_1) \rangle
\end{equation}

As the maximum eigenvalue is less sensitive with respect to numerical errors, it is recommended to use the dominant eigenvectors $ \xi_2, \eta_2 $ in order to compute $ PRA_{t_0}^{t_1}(\mathbf{x}_0) $.

In [3]:
    def _PRA_(self):
            
        if hasattr(self, 'grad_Fmap_grid') == False:
        
            self._grad_Fmap_grid()
            
        self.PRA = np.zeros(self.X_domain.shape)
            
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
                
                U, S, V = self._svd(self.grad_Fmap_grid[i, j, :, :])
                
                self.PRA[i, j] = np.arccos(U[0, 0]*V[0, 0]+U[0, 1]*V[0, 1])
        
        return self.PRA

<a id='LAVD'></a>
### Lagrangian vortex boundary from PRA

In [4]:
    def _vortex_boundary_and_center_from_PRA(self, n_contours = 100, max_distance = .5, convexity_threshold = 10**(-5), length_threshold = .5):
        
        contours = self._contourlines(self.X_domain, self.Y_domain, self.PRA, n_contours)
        x_max, y_max, LAVD_max = self._find_2D_peaks(max_distance, self.X_domain, self.Y_domain, self.PRA)
        
        vortex_boundary = []
        vortex_center_x, vortex_center_y = [], []
        
        for i in range(len(x_max)):
            
            tubular_surfaces = []
            length_tubes = []
            vortex_center_x_candidate, vortex_center_y_candidate = [], []
            
            for c in contours:
            
                bool_contour = self._point_inside_polygon(x_max[i], y_max[i], c)
                
                if bool_contour:
                    
                    convexity_deficiency = self._convexity_deficiency(c)
                    length = self._length_closed_material_line(c)
                    
                    if convexity_deficiency < convexity_threshold and length > length_threshold:
                        
                        tubular_surfaces.append(c)
                        length_tubes.append(length)
                        vortex_center_x_candidate.append(x_max[i])
                        vortex_center_y_candidate.append(y_max[i])
            
            if len(length_tubes) > 0:           
                argmax_length = np.argmax(length_tubes)
                vortex_boundary.append(tubular_surfaces[argmax_length])
                vortex_center_x.append(vortex_center_x_candidate[argmax_length])
                vortex_center_y.append(vortex_center_y_candidate[argmax_length])           

        return vortex_boundary, vortex_center_x, vortex_center_y

## Lagrangian Averaged Vorticity Deviation (LAVD)

In [5]:
    def _LAVD(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()
            
        def parallelization(k, t):
                
            self.omega = self._vorticity(t)
                
            spatially_averaged_vorticity = np.nanmean(self.omega.ravel())  
                
            LVD = np.zeros((self.len_Y, self.len_X))
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    x = np.array([self.trajectory_grid[i, j, 0, k], self.trajectory_grid[i, j, 1, k]])

                    W = self._vorticity_tensor(x, t)
                    
                    omega = W[0, 1]-W[1, 0]
                
                    LVD[i, j] = np.abs(omega-spatially_averaged_vorticity)
                    
            return LVD
        
        self.LVD = np.array(Parallel(n_jobs=self.Ncores, verbose = 1)(delayed(parallelization)(k, t) for k, t in tqdm(enumerate(self.time), total=self.lenT)))
        
        self.LAVD = np.nanmean(self.LVD, axis = 0)
        
        return self.LAVD

### Lagrangian vortex boundary from LAVD

In [6]:
    def _vortex_boundary_and_center_from_LAVD(self, n_contours = 100, max_distance = .5, convexity_threshold = 10**(-5), length_threshold = .5):
        
        contours = self._contourlines(self.X_domain, self.Y_domain, self.LAVD, n_contours)
        x_max, y_max, LAVD_max = self._find_2D_peaks(max_distance, self.X_domain, self.Y_domain, self.LAVD)
        
        vortex_boundary = []
        vortex_center_x, vortex_center_y = [], []
        
        for i in range(len(x_max)):
            
            tubular_surfaces = []
            length_tubes = []
            vortex_center_x_candidate, vortex_center_y_candidate = [], []
            
            for c in contours:
            
                bool_contour = self._point_inside_polygon(x_max[i], y_max[i], c)
                
                if bool_contour:
                    
                    convexity_deficiency = self._convexity_deficiency(c)
                    length = self._length_closed_material_line(c)
                    
                    if convexity_deficiency < convexity_threshold and length > length_threshold:
                        
                        tubular_surfaces.append(c)
                        length_tubes.append(length)
                        vortex_center_x_candidate.append(x_max[i])
                        vortex_center_y_candidate.append(y_max[i])
            
            if len(length_tubes) > 0:           
                argmax_length = np.argmax(length_tubes)
                vortex_boundary.append(tubular_surfaces[argmax_length])
                vortex_center_x.append(vortex_center_x_candidate[argmax_length])
                vortex_center_y.append(vortex_center_y_candidate[argmax_length])      

        return vortex_boundary, vortex_center_x, vortex_center_y

<a id='Quasi_objective_diagnostics'></a>
## Quasi-Objective diagnostics

<a id='TRA'></a>
### Trajectory Rotation Average (TRA)

In [7]:
    def _TRA_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()
        
        self.TRA = np.zeros((self.len_Y, self.len_X))
            
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                    
                velx0 = self.velocity_grid[i, j, 0, 0]
                vely0 = self.velocity_grid[i, j, 1, 0]
                    
                vel0 = np.sqrt(velx0**2+vely0**2)
                    
                velxN = self.velocity_grid[i, j, 0, -1]
                velyN = self.velocity_grid[i, j, 1, -1]
                
                vel1 = np.sqrt(velxN**2+velyN**2)
                
                self.TRA[i, j] = np.abs(np.arccos((velx0*velxN+vely0*velyN)/(vel0*vel1)))
        
        return self.TRA

<a id='TRA_bar'></a>
### Trajectory Rotation Average without cancellations $ (\overline{TRA}) $

In [8]:
    def _TRA_bar_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()
        
        self.TR_bar = np.zeros((self.len_Y, self.len_X, self.lenT-2))
        
        for k in tqdm(range(self.lenT-2), total = self.lenT-2):
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    velx0 = self.velocity_grid[i, j, 0, k]
                    vely0 = self.velocity_grid[i, j, 1, k]
                    
                    vel0 = np.sqrt(velx0**2+vely0**2)
                    
                    velx1 = self.velocity_grid[i, j, 0, k + 1]
                    vely1 = self.velocity_grid[i, j, 1, k + 1]
                
                    vel1 = np.sqrt(velx1**2+vely1**2)
                
                    self.TR_bar[i, j, k-1] = np.abs(np.arccos((velx0*velx1+vely0*vely1)/(vel0*vel1)))
        
        self.TRA_bar = np.nanmean(self.TR_bar, axis = 2)/(self.tN-self.t0)
        
        return self.TRA_bar

<a id='TSE'></a>
### Trajectory Stretching Exponent $ (TSE) $

In [9]:
    def _TSE_(self):
    
        if hasattr(self, 'trajectory_grid') == False:
            
            self._trajectory_grid()
        
        self.TSE = np.zeros((self.len_Y, self.len_X))
            
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                    
                velx0 = self.velocity_grid[i, j, 0, 0]
                vely0 = self.velocity_grid[i, j, 1, 0]
                    
                vel0 = np.sqrt(velx0**2+vely0**2)
                    
                velxN = self.velocity_grid[i, j, 0, -1]
                velyN = self.velocity_grid[i, j, 1, -1]
                
                velN = np.sqrt(velxN**2+velyN**2)
                
                self.TSE[i, j] = 1/(self.tN-self.t0)*np.log(velN/vel0)
        
        return self.TSE

<a id='TSE_bar'></a>
### Trajectory Stretching Exponent without cancellations $ (\overline{TSE}) $

In [10]:
    def _TSE_bar_(self):
        
        if hasattr(self, 'trajectory_grid') == False:
            
            raise Exception("You need to calculate trajectories over meshgrid and then compute velocity using finite differencing!")
        
        self.TSE_bar = np.zeros((self.len_Y, self.len_X, self.lenT-2))
        
        for k in tqdm(range(self.lenT-2), total = self.lenT-2):
            
            for i in range(self.len_Y):
            
                for j in range(self.len_X):
                    
                    velx0 = self.velocity_grid[i, j, 0, k]
                    vely0 = self.velocity_grid[i, j, 1, k]
                    
                    vel0 = np.sqrt(velx0**2+vely0**2)
                    
                    velx1 = self.velocity_grid[i, j, 0, k + 1]
                    vely1 = self.velocity_grid[i, j, 1, k + 1]
                
                    vel1 = np.sqrt(velx1**2+vely1**2)
                
                    self.TSE_bar[i, j, k] = np.abs(np.log(vel1/vel0))
        
        self.TSE_bar = np.nanmean(self.TSE_bar, axis = 2)/(self.tN-self.t0)
        
        return self.TRA_bar

<a id='Analytic_methods'></a>
# Analytic methods

<a id='Hyperbolic_LCS'></a>
## Hyperbolic LCS

<a id='local_variational_theory_hyperbolic'></a>
###  Hyperbolic LCS from local variational theory

In [11]:
    def _hyperbolic_LCS_local_variational_theory(self, s_array = np.linspace(0, 20, 100), lamda_threshold = 1, max_distance = 1, tensorline = "shrinklines", max_line_length = 5):
        
        bool_forward = True
        bool_backward = True
        
        # Define step-size to be used for integration
        step_size = s_array[1]-s_array[0]
        
        # Create gridded array which store the eigenvalue and eigenvectors of the cauchy-green strain tensor
        eigenvalue_max = np.zeros((self.len_Y, self.len_X))
        eigenvector_max = np.zeros((self.len_Y, self.len_X, self.dim))
        eigenvalue_min = np.zeros((self.len_Y, self.len_X))
        eigenvector_min = np.zeros((self.len_Y, self.len_X, self.dim))
        
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
        
                lambda_min, lambda_max, v_min, v_max = self._eigenvalues_and_eigenvectors(self.C[i, j, :, :])
        
                # Make sure that lambda_max and lambda_min are finite and not nan.
                # Otherwise the gridded interpolation would not work.
                if np.isfinite(lambda_max) and np.isfinite(lambda_min):
                
                    eigenvalue_max[i, j] = lambda_max
                    eigenvalue_min[i, j] = lambda_min
                    eigenvector_max[i, j, :] = v_max
                    eigenvector_min[i, j, :] = v_min

        # Repelling LCS (=forward shrinklines)
        if tensorline == "shrinklines":
    
            lamda = eigenvalue_max
            vector_field = eigenvector_min
            
        # Attracting LCS (=forward stretchlines)
        elif tensorline == "stretchlines":
    
            lamda = eigenvalue_min
            lamda_threshold = -lamda_threshold
            vector_field = eigenvector_max
        
        else:    
            print("Variable type should either be shrinklines or stretchlines")
        
        # Find local extrema of eigenvalue field
        peak_x, peak_y, peak_field = self._find_2D_peaks(max_distance, self.X_domain, self.Y_domain, lamda)
        
        self.Interp_lambda = self._gridded_Interpolation(self.Y_domain, self.X_domain, lamda, "linear")
        self.Interp_lambda_max = self._gridded_Interpolation(self.Y_domain, self.X_domain, eigenvalue_max, "linear")
        self.Interp_lambda_min = self._gridded_Interpolation(self.Y_domain, self.X_domain, eigenvalue_min, "linear")
    
        x_tensorlines, y_tensorlines = [], []
        lambda_tensorlines = []
        
        for i in tqdm(range(len(peak_x))):
            
            x_tensorline_forw, y_tensorline_forw = [], []
            x_tensorline_back, y_tensorline_back = [], []
            lambda_tensorline_forw, lambda_tensorline_back = [], []
            
            x = np.array([peak_x[i], peak_y[i]])
            bool_forward, bool_backward = True, True
            
            for j in range(len(x_tensorlines)):
                
                for k in range(len(x_tensorlines[j])):
                    
                    if np.sqrt((x[0]-x_tensorlines[j][k])**2+(x[1]-y_tensorlines[j][k])**2) < max_distance:
                        bool_forward = False
                        bool_backward = False
                        break
            
            if bool_forward and bool_backward:
                
                x_forward = x
                x_backward = x
                
                vx, vy = self._orientational_discontinuity(x, vector_field)
                
                x_tensorline_forw.append(x[0])
                y_tensorline_forw.append(x[1])
                    
                x_prime_forward = np.array([vx, vy])
                x_prime_backward = -np.array([vx, vy])
                
                dist = 0
            
                for s in range(len(s_array)):
                    
                    if bool_forward and x_prime_forward is not None:
                        x_old_forw = x_forward
                        x_forward, x_prime_forward = self._RK4_tensorlines_orientational_discontinuity(x_forward, x_prime_forward, step_size, vector_field) 
                        if x_forward is not None:
                        
                            dist += np.sqrt((x_forward[0]-x_old_forw[0])**2+(x_forward[1]-x_old_forw[1])**2)
                            lamda_dummy = self.Interp_lambda(x_forward[1], x_forward[0])
                        
                            if dist < max_line_length and lamda_dummy > lamda_threshold and (np.min(self.X_domain) < x_forward[0] < np.max(self.X_domain)) and (np.min(self.Y_domain) < x_forward[1] < np.max(self.Y_domain)):
                                x_tensorline_forw.append(x_forward[0])
                                y_tensorline_forw.append(x_forward[1])
                                lambda_tensorline_forw.append(lamda_dummy)
                    
                        else:
                        
                            bool_forward = False
                            
                    if bool_backward and (x_prime_forward is not None):
                        x_old_back = x_backward
                        x_backward, x_prime_backward = self._RK4_tensorlines_orientational_discontinuity(x_backward, x_prime_backward, step_size, vector_field) 
                        if x_backward is not None:
                        
                            dist += np.sqrt((x_backward[0]-x_old_back[0])**2+(x_backward[1]-x_old_back[1])**2)
                            lamda_dummy = self.Interp_lambda(x_backward[1], x_backward[0])
                            
                            if dist < max_line_length and lamda_dummy > lamda_threshold and (np.min(self.X_domain) < x_backward[0] < np.max(self.X_domain)) and (np.min(self.Y_domain) < x_backward[1] < np.max(self.Y_domain)):
                                x_tensorline_back.append(x_backward[0])
                                y_tensorline_back.append(x_backward[1])
                                lambda_tensorline_back.append(lamda_dummy)
                    
                        else:
                        
                            bool_backward = False
                        
                x_tensorline = np.append(np.flip(x_tensorline_back), x_tensorline_forw)
                y_tensorline = np.append(np.flip(y_tensorline_back), y_tensorline_forw)
            
                x_tensorlines.append(x_tensorline)
                y_tensorlines.append(y_tensorline)
                lambda_tensorlines.append(np.mean(np.append(np.flip(lambda_tensorline_back), lambda_tensorline_forw)))
            
        return x_tensorlines, y_tensorlines, lambda_tensorlines, peak_x, peak_y

#### Tensorlines - autonomous differential equation

In [12]:
    def _differential_system_tensorlines_orientational_discontinuity(self, x, x_prime, vector_field):
        
        vx, vy = self._orientational_discontinuity(x, vector_field)
        
        if vx is not None:
        
            lambda_max = self.Interp_lambda_max(x[1], x[0])[0][0]
            lambda_min = self.Interp_lambda_min(x[1], x[0])[0][0]
        
            alpha = ((lambda_max-lambda_min)/(lambda_max+lambda_min))**2
            
            scaling = np.sign(vx*x_prime[0]+vy*x_prime[1])*alpha
            
            return scaling*np.array([vx, vy])
        
        else:
            return None

#### Orientational discontinuity

In [13]:
    def _orientational_discontinuity(self, x, vector_field):
        
        # Check for orientational discontinuity by introducing appropriate scaling
        idx_x = np.searchsorted(self.X_domain[0,:], x[0])
        idx_y = np.searchsorted(self.Y_domain[:,0], x[1])
        
        if 0 < idx_x < self.X_domain.shape[1] and 0 < idx_y < self.X_domain.shape[0]:
    
            X, Y = self.X_domain[idx_y-1:idx_y+1, idx_x-1:idx_x+1], self.Y_domain[idx_y-1:idx_y+1, idx_x-1:idx_x+1]
    
            vx_grid = np.array([[vector_field[idx_y-1,idx_x-1, 0], vector_field[idx_y, idx_x-1, 0]],
                      [vector_field[idx_y-1,idx_x, 0], vector_field[idx_y, idx_x, 0]]])
            vy_grid = np.array([[vector_field[idx_y-1,idx_x-1, 1], vector_field[idx_y, idx_x-1, 1]],
                      [vector_field[idx_y-1,idx_x, 1], vector_field[idx_y, idx_x, 1]]])
        
            for i in range(self.dim):
                for j in range(self.dim):
                    if vx_grid[0, 0]*vx_grid[i, j]+vy_grid[0, 0]*vy_grid[i, j] < 0:
                        vx_grid[i, j] = -vx_grid[i, j]
                        vy_grid[i, j] = -vy_grid[i, j]
    
            vx_Interp = self._gridded_Interpolation(Y, X, vx_grid, "linear")
            vy_Interp = self._gridded_Interpolation(Y, X, vy_grid, "linear")
        
            return vx_Interp(x[1], x[0])[0][0], vy_Interp(x[1], x[0])[0][0]
        
        else:
            
            return None, None

#### Steady $ 4^{th} $-order Runge - Kutta Integration handling orientational discontinuities

In [14]:
    def _RK4_tensorlines_orientational_discontinuity(self, x, x_prime, ds, vector_field):
        
        # Define starting point.
        x1 = x
        
        # If x is outside defined domain --> vel is None --> _RK4 returns "None" and integration will stop.
        
        loc = self._check_location(x1)[0]
        
        if loc != "IN" or self.Interp_lambda_max(x1[1], x1[0])[0][0] < 1:
            return None, None
        
        # Compute x_prime at the beginning of the time-step
        x_prime = self._differential_system_tensorlines_orientational_discontinuity(x1, x_prime, vector_field)
        if x_prime is None:
            return None, None
        k1 = ds * x_prime

        #  position and time at the first midpoint.
        x2 = x1 + .5 * k1
        loc = self._check_location(x2)[0]
        
        if loc != "IN" or self.Interp_lambda_max(x2[1], x2[0])[0][0] < 1:
            return None, None
        
        # Compute x_prime at the first midpoint.
        x_prime = self._differential_system_tensorlines_orientational_discontinuity(x2, x_prime, vector_field)
        if x_prime is None:
            return None, None
        k2 = ds * x_prime

        # Update position at the second midpoint.
        x3 = x1 + .5 * k2
    
        loc = self._check_location(x3)[0]
        if loc != "IN" or self.Interp_lambda_max(x3[1], x3[0])[0][0] < 1: 
            return None, None
    
        # Compute velocity at the second midpoint.
        x_prime = self._differential_system_tensorlines_orientational_discontinuity(x3, x_prime, vector_field)
        if x_prime is None:
            return None, None
        k3 = ds * x_prime
    
        # Update position at the endpoint.
        x4 = x1 + k3
    
        loc = self._check_location(x4)[0]
        if loc != "IN" or self.Interp_lambda_max(x4[1], x4[0])[0][0] < 1:
            return None, None
    
        # Compute velocity at the end of the time-step.
        x_prime = self._differential_system_tensorlines_orientational_discontinuity(x4, x_prime, vector_field) 
        if x_prime is None:
            return None, None
        k4 = ds * x_prime
    
        # define list for velocity and positions of particle
        x_prime_update = []
        x_update = []
        
        # Compute velocity
        for j in range(self.dim):
            # Update velocity of particles
            x_prime_update.append(1.0 / 6.0*(k1[j] + 2 * k2[j] + 2 * k3[j] + k4[j])/ds)
    
        # Integration x <-- x + x_prime*ds
        for j in range(self.dim):
            # Update position of particles
            x_update.append(x[j] + x_prime_update[j]*ds)

        x_update = np.array(x_update)
        x_prime_update = np.array(x_prime_update)
        
        if self._check_location(x_update)[0] != "IN" or self.Interp_lambda_max(x_update[1], x_update[0])[0][0] < 1: 
            return None, None
    
        return x_update, x_prime_update

<a id='elliptic_LCS'></a>
## Elliptic LCS

<a id='local_variational_theory_elliptic'></a>
### Elliptic LCS from local variational theory

In [15]:
    def _elliptic_LCS_local_variational_theory(self, s_array = np.linspace(0, 20, 100), len_poincare = 3):
        
        bool_elliptic = True
        
        # Shearlines
        self.shear_vector = np.zeros((self.len_Y, self.len_X))
        
        # Step-size to be used for integration
        step_size = s_array[1]-s_array[0]
        
        for i in range(self.len_Y):
        
            for j in range(self.len_X):
        
                lambda_min, lambda_max, v_min, v_max = self._eigenvalues_and_eigenvectors(self.C[i, j, :, :])
        
                # Make sure that lambda_max and lambda_min are finite and not nan.
                # Otherwise the gridded interpolation would not work.
                if np.isfinite(lambda_max) and np.isfinite(lambda_min):
                    
                    denom = np.sqrt(lambda_min)+np.sqrt(lambda_max)
                
                    self.shear_vector[i, j, :] = np.sqrt(np.sqrt(lambda_max)/denom)*v_min+np.sqrt(np.sqrt(lambda_min)/denom)*v_max
        
        self._singularity()

### Numerical detection of singularities

In [16]:
    def _singularity(self):
        
        f = self.C[:, :, 0, 0]-self.C[:, :, 1, 1]
        g = self.C[:, :, 0, 1]
    
        x_singularity, y_singularity = [], []
        
        for i in range(self.len_Y-1):
            
            for j in range(self.len_X-1):
                
                f_0 = np.max(f[i:i+2, j:j+2].ravel()) >= 0 and np.min(f[i:i+2, j:j+2].ravel()) <= 0
                g_0 = np.max(g[i:i+2, j:j+2].ravel()) >= 0 and np.min(g[i:i+2, j:j+2].ravel()) <= 0
                
                lamda_min, lamda_max, _, _ = self._eigenvalues_and_eigenvectors(self.C[i, j, :, :])
                
                # 0.9 < det(C) < 1.1  
                det_C = self.C[i, j, 0, 0]*self.C[i, j, 1, 1]-self.C[i, j, 1, 0]*self.C[i, j, 0, 1]
                filter_singularity = .8 < det_C < 1.2
                
                if f_0 and g_0 and filter_singularity:
                    
                    x, y = self.X_domain[i, j], self.Y_domain[i, j]
                    
                    x_singularity.append(self.X_domain[i, j]+self.dx/2)
                    y_singularity.append(self.Y_domain[i, j]+self.dy/2)
        
        return x_singularity, y_singularity

In [17]:
    def _singularity_type(self, x, y, poincare_length, n_points):
        
        for i in range(0, poincare_length, n_points):
            
            f_theta = []

            for theta in range(0, 2*np.pi):
                
                x_circle = x+i*np.cos(theta)
                y_circle = y+i*np.sin(theta)
                
                xi_1 = self._orientational_discontinuity(x)
                
                if xi_1 is None:
                    
                    break
                
                else:
                
                    xi_1_abs = np.sqrt(xi_1[0]**2+xi_1[1]**2)
                
                    f_theta.append(abs(xi_1[0]*i*np.cos(theta)+xi_1[1]*i*np.sin(theta))/(xi_1_abs*i))
        
            import matplotlib.pyplot as plt
        
            plt.plot(f_theta)
            plt.show()

### Poincare section

In [18]:
    def _poincare_section(self, x_singularity, y_singularity):
        
        pass

In [19]:
    def _hyperbolic_LCS_global_variational_theory(self):
        
        print("=================Hyperbolic LCS from global variational theory=================")

In [20]:
    def _elliptic_LCS_global_variational_theory(self):
        
        print("=================Elliptic LCS from global variational theory=================")

In [21]:
    def _vorticity(self, t):
        
        self.omega = np.zeros((self.len_Y, self.len_X))
        
        for i in range(self.len_Y):
            
            for j in range(self.len_X):
                
                x = np.array([self.X_domain[i, j], self.Y_domain[i, j]])
    
                W = self._vorticity_tensor(x, t)
                
                self.omega[i, j] = W[0, 1]-W[1, 0]
                
        return self.omega              

# General functions

## Ridge extraction

In [22]:
    def _find_ridges(self, Field, threshold = None, method = "threshold", resolution = 1, ds = 1, n_iterations = 100):
        
        sign = 1
            
        if threshold is None:
            
            print("Threshold value is None --> Specify threshold.")
            print("If not specified, threshold is set to ", 0.1, " of the maximum value of the given scalar field")
            
            threshold = .5*np.nanmax(Field)
            
        if method == "threshold":
            
            mask = (Field >= threshold)
            
            extrema_x = self.X_domain[mask].ravel()
            extrema_y = self.Y_domain[mask].ravel()
            
            return extrema_x, extrema_y
            
        elif method == "gradient":
            
            Field_nan = Field.copy()
            
            Field[np.isnan(Field)] = 0
            
            Interpolant_Field = self._gridded_Interpolation(self.Y_domain, self.X_domain, Field, method = "cubic")
            
            grad_Field = np.zeros((Field.shape[0], Field.shape[1], 2))
            
            for i in range(1, Field.shape[0]-1):
                
                for j in range(1, Field.shape[1]-1):
                    
                    dy = (self.Y_domain[i+1,0] - self.Y_domain[i-1, 0])/2
                    dx = (self.X_domain[0, j+1] - self.X_domain[0, j-1])/2

                    grad_Field[i, j, 0] = (Interpolant_Field(self.Y_domain[i, j], self.X_domain[i, j]+.1*dx)[0][0]-Interpolant_Field(self.Y_domain[i, j], self.X_domain[i, j]-.1*dx)[0][0])/(2*.1*dx)
                    grad_Field[i, j, 1] = (Interpolant_Field(self.Y_domain[i, j]+.1*dy, self.X_domain[i, j])[0][0]-Interpolant_Field(self.Y_domain[i, j]-.1*dy, self.X_domain[i, j])[0][0])/(2*.1*dy)
        
            grad_Fieldx = grad_Field[:, :, 0]
            grad_Fieldy = grad_Field[:, :, 1]
            grad_Fieldx[np.isnan(grad_Fieldx)] = 0
            grad_Fieldy[np.isnan(grad_Fieldy)] = 0
            
            Interpolant_gradx_Field = self._gridded_Interpolation(self.Y_domain, self.X_domain, grad_Fieldx)
            Interpolant_grady_Field = self._gridded_Interpolation(self.Y_domain, self.X_domain, grad_Fieldy)
            
            x_grid = np.linspace(np.min(self.X_domain), np.max(self.X_domain), self.Y_domain.shape[0]*resolution)
            y_grid = np.linspace(np.min(self.Y_domain), np.max(self.Y_domain), self.Y_domain.shape[1]*resolution)
            
            extrema_x = []
            extrema_y = []
            
            for x in tqdm(x_grid, total = len(x_grid)):
                    
                for y in y_grid:
                    
                    x_eval = x
                    y_eval = y
    
                    loc = self._check_location(np.array([x_eval, y_eval]))[0]
                
                    gradient = 10
                    
                    iter = 0
                        
                    while iter < n_iterations and loc == "IN" and Interpolant_Field(y_eval, x_eval)[0][0] > threshold:
                        
                        gradx = Interpolant_gradx_Field(y_eval, x_eval)[0][0]
                        grady = Interpolant_grady_Field(y_eval, x_eval)[0][0]
                        
                        gradient = np.sqrt(gradx**2+grady**2)
                        
                        x_eval = x_eval + sign * ds * gradx/gradient*dx
                        y_eval = y_eval + sign * ds * grady/gradient*dy
                        
                        iter += 1
                        
                        if np.min(self.X_domain) < x_eval < np.max(self.X_domain) and np.min(self.Y_domain) < y_eval < np.max(self.Y_domain):
                        
                            loc = self._check_location(np.array([x_eval, y_eval]))[0]
                            
                        else:
                            
                            loc = "OUT"
                        
                        idx_x = np.searchsorted(self.X_domain[0,:], x_eval)
                        idx_y = np.searchsorted(self.Y_domain[:,0], y_eval)
                        
                        if 0 < idx_x < self.len_X-1 and 0 < idx_y < self.len_Y-1:
                        
                            if iter == n_iterations and loc == "IN" and np.isfinite(np.sum(Field_nan[idx_y-1:idx_y+1,idx_x-1:idx_x+1])):
                    
                                extrema_x.append(x_eval)
                                extrema_y.append(y_eval)
            
            return extrema_x, extrema_y

        else:
            
            print("The method argument is not valid. Use either threshold or gradient")

## Local extrema

In [23]:
    def _find_2D_peaks(self, max_distance, X, Y, Field):
        
        def _find_all_local_maxima(X, Y, Field):
            
            loc_max_x, loc_max_y, loc_max_field = [], [], []
            
            for i in tqdm(range(2, X.shape[0]-2)):
                
                for j in range(2, Y.shape[1]-2):
                    
                    if np.isfinite(Field[i, j]) and Field[i, j] > Field[i+1, j] and Field[i, j] > Field[i-1, j] and Field[i, j] > Field[i, j+1] and Field[i, j] > Field[i, j-1]:
                        
                        loc_max_x.append(X[i, j])
                        loc_max_y.append(Y[i, j])
                        loc_max_field.append(Field[i, j])
            
            return loc_max_x, loc_max_y, loc_max_field
        
        loc_max_x, loc_max_y, loc_max_field = _find_all_local_maxima(X, Y, Field)
        
        n_loc_max = len(loc_max_x)
        
        peak_x, peak_y, peak_field = [], [], []
        
        for i in range(n_loc_max):
            
            bool_loc_max = True
    
            for j in range(n_loc_max):
            
                if i != j and loc_max_field[i] < loc_max_field[j] and np.sqrt((loc_max_x[i]-loc_max_x[j])**2+(loc_max_y[i]-loc_max_y[j])**2) <= max_distance:
                    
                    bool_loc_max = False
                
            if bool_loc_max:
                
                peak_x.append(loc_max_x[i])
                peak_y.append(loc_max_y[i])
                peak_field.append(loc_max_field[i])
          
        arg_index = np.argsort(peak_field)
        peak_x = np.flip([peak_x[i] for i in arg_index])
        peak_y = np.flip([peak_y[i] for i in arg_index])
        peak_field = np.flip([peak_field[i] for i in arg_index])
    
        return peak_x, peak_y, peak_field

## Contourlines

In [24]:
    def _contourlines(self, X, Y, Field, n_contours):
        
        from skimage import measure
        from shapely.geometry import Polygon, Point
        
        Field_min = np.nanmin(Field) + (np.nanmax(Field)-np.nanmin(Field))/n_contours
        Field_max = np.nanmax(Field) - (np.nanmax(Field)-np.nanmin(Field))/n_contours
        
        closed_contours = []
        
        for i in np.linspace(Field_min, Field_max, n_contours):
            
            contour = measure.find_contours(Field, i)
            
            for c in contour:
                
                if c[-1, 0] == c[0, 0] and c[-1, 1] == c[0, 1]:

                    x_contour = np.min(X)+(np.max(X)-np.min(X))/(X.shape[1])*c[:, 1]
                    y_contour = np.min(Y)+(np.max(Y)-np.min(Y))/(Y.shape[0])*c[:, 0]
            
                    closed_contours.append(np.array([x_contour, y_contour]).T)
    
        return closed_contours

## Area of polygon

In [25]:
    def polygonArea(self, X, Y):

        # Initialize area
        area = 0.0
 
        # Calculate value of shoelace formula
        j = len(X) - 1
        for i in range(0,len(X)):
            area += (X[j] + X[i]) * (Y[j] - Y[i])
            j = i # j is previous vertex to i
        
        return abs(area / 2.0)

## Convexity deficiency

In [26]:
    def _convexity_deficiency(self, contours):

        from scipy.spatial import ConvexHull, convex_hull_plot_2d
        from shapely.geometry import Polygon, Point
            
        X = contours[:, 0]
        Y = contours[:, 1]
            
        points = np.array([X, Y]).T
            
        Polygon_area = self.polygonArea(X, Y)
            
        if Polygon_area > 0:
                
            hull = ConvexHull(points)
            
            Convex_area = hull.volume
            
            return (Convex_area-Polygon_area)/Polygon_area
                
        else:
                
            return np.nan

## Point inside polygon

In [27]:
    def _point_inside_polygon(self, x, y, contour):
        
        from shapely.geometry import Point
        from shapely.geometry.polygon import Polygon
        
        point = Point(x, y)
        polygon = Polygon(contour)
        
        return polygon.contains(point)

## Length closed material line

In [28]:
    def _length_closed_material_line(self, contour):
        
        from shapely.geometry.polygon import Polygon
        
        length = 0
        
        for i in range(contour.shape[0]-1):
            
            length += np.sqrt((contour[i+1,0]-contour[i,0])**2+(contour[i+1,1]-contour[i,1])**2)
        
        return length